# Test notebook for the workshop

*Author : Frédéric FERRY - Météo-France / Ecole Nationale de la Météorologie (June 2023)*

Run the cell below. If all modules load without any error message ("ModuleNotFoundError") and if an animation runs in the last cell, this should work fine for the rest of the workshop.

In [ ]:
import os
import time

from PIL import Image
from IPython.display import Image as IPImage
from IPython.display import display

from tqdm import tqdm

import math

import xarray as xr
import netCDF4
import numpy as np
import pandas as pd

from shapely.geometry import Point

import metpy.calc as mpcalc
from metpy.units import units

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.collections as collections

from scipy.ndimage import gaussian_filter, maximum_filter, minimum_filter

from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

from eofs.standard import Eof

In [ ]:
def make_animation(gif_filepath):
    
    image_folder = './anim/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

In [ ]:
projection=ccrs.PlateCarree()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines()
    ax.set_xticks(np.linspace(-180, 180, 37), crs=ccrs.PlateCarree())
    ax.set_yticks(np.linspace(-90, 90, 37), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return(ax)

In [ ]:
def plot_maxmin_points(data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values,
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

def print_maxmin_points(data, extrema, nsize):
    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        # Print date lon lat and pressure of the minimum
        print(date, data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, int(data[mxy[i], mxx[i]]))

In [ ]:
Ana_MSLP = './data/MSLP_September2018_HiRes.nc'
Ana_qT850 = './data/qT850_September2018_HiRes.nc'

t0 ='2018-09-24T00'
t1 ='2018-09-30T12'

data_mslp   = xr.open_dataset(Ana_MSLP).sel(time=slice(t0,t1))
data_qT   = xr.open_dataset(Ana_qT850).sel(time=slice(t0,t1))

print(data_mslp)

In [ ]:
mslp_ana = data_mslp['msl']/100
q_ana = data_qT['q']
T_ana = data_qT['t']

lats = mslp_ana.latitude.values
lons = mslp_ana.longitude.values

time = mslp_ana.time.values
times_str=[x for x in range(len(time))]
dates_str=[x for x in range(len(time))]
for i in range(len(time)):
	times_str[i] = str(time[i])
	dates_str[i] = times_str[i][0:13]
print(dates_str)

In [ ]:
Td = mpcalc.dewpoint_from_specific_humidity(850* units.hPa, T_ana.values* units.kelvin, q_ana.values* units('kg/kg'))
Thetae_ana=mpcalc.equivalent_potential_temperature(850* units.hPa, T_ana, Td)
print(Thetae_ana.shape)
print(np.min(Thetae_ana))
print(np.max(Thetae_ana))

In [ ]:
cmap='jet'
clevs1 = np.linspace(5500, 6000, 21)
clevs2 = np.arange(994, 1040, 2)
clevs3 = np.linspace(1, 12, 21)
clevs4 = np.arange(270,345,5)
clevs5=np.arange(-30,2.5,2.5)

for i in tqdm(range(len(time))):
    
    #print(dates_str[i])

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle('Analysis : '+dates_str[i], fontsize=14)
    
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
    plt.title('MSLP and $\Theta_E$ at 850 hPa', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lons, lats, Thetae_ana[i,:,:], levels=clevs4, cmap=cmap, extend='both',transform=ccrs.PlateCarree())
    c = ax.contour(lons, lats, mslp_ana[i,:,:], levels=clevs2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
    cb.set_label('K', fontsize=12)
    
    plt.close()
    figname='./anim/MSLP_Thetae850_'+dates_str[i]
    fig.savefig(figname+'.png')

In [ ]:
gif_filepath = './anim/MSLP_Thetae850_an.gif'
make_animation(gif_filepath)